# Escaneando

Esta es la segunda fase del bloque de reconocimiento. Tras haber recopilado toda la información posible sobre el objetivo, en esta fase intentaremos descubrir qué equipos del cliente tiene expuestos en internet (escaneo de red), qué servicios tienen habilitados esos equipos (escaneo de puertos) y si esos equipos y servicios habilitados tienen vulnerabilidades conocidas (escaneo de vulnerabilidades). El objetivo de esta fase es utilizar toda la información que hemos encontrado en la fase anterior para detectar vectores de ataque y vías de entrada en la red que se este auditando.

### Metodología

Se recomienda seguir usando [cherrytree](https://www.giuspen.com/cherrytree/) para ir guardando las evidencias.

1. <a style="color:#000;text-decoration:none;cursor:pointer" href="#intro">Breve introducción informal</a>
  * ICMPv4
  * Cabecera del protocolo TCP
<br><br>
2. <a style="color:#000;text-decoration:none;cursor:pointer" href="#escaneando-red">Escaneando la red para descubrir equipos activos</a>
  * Detectar si un equipo esta activo usando ping
  * Ping Sweep/ICMP Sweep detectar los equipos activos en una red
  * Ping Sweep/ICMP Sweep usando nmap
  * Detectar equipos activos usando netdiscover
  * Descubriendo equipos activos por TCP usando hping3
<br><br>
3. <a style="color:#000;text-decoration:none;cursor:pointer" href="#detectando-servicios">Escaneando los puertos de los equipos activos</a>
  * Equipo para realizar escaneos de pruebas (scanme.nmap.org)
  * Escaneando equipos desactivando el host discovery
  * Escaneo con handshake completo (fiable pero ruidoso)
  * Escaneo con medio handshake (SYN Scan)
  * Escaneo detectando la versión de los servicios
  * Escaneo de medio handshake detectando la versión de los servicios
  * Escaneo de servicios en puertos UDP
  * Detectando el sistema operativo
    * nmap
    * p0f
  * Nmap Scripting Engine (NSE)
  * Escaneo agresivo con nmap
  * Escaneo ACK para descubrir dispositivos de seguridad
    * nmap
    * hping3
<br><br>

4. <a style="color:#000;text-decoration:none;cursor:pointer" href="#proxies">Proxies y programas que ayudan proteger la privacidad e intimidad</a>
  * Breve introducción informal a los conceptos de proxy y cadena de proxies
  * Escaneo con nmap a través de TOR usando proxychains
  * Hackplayers - 4nonimizer
<br><br>

5. <a style="color:#000;text-decoration:none;cursor:pointer" href="#detectando-vulnerabilidades">Escaneando en busca de vulnerabilidades conocidas</a>
  * Breve introducción informal sobre los escáneres de vulnerabilidades
  * Breve introducción informal sobre el análisis de vulnerabilidades
  * Breve introducción al análisis de vulnerabilidades con OpenVas
<br><br>

### Referencias usadas

1. Páginas web
  * [OSI](https://es.wikipedia.org/wiki/Modelo_OSI)
  * [Nmap examples](https://nmap.org/man/es/man-port-scanning-techniques.html)
  * [Nmap Host discovery](https://nmap.org/book/man-host-discovery.html)
  * [Nmap port scanning techniques](https://nmap.org/man/es/man-port-scanning-techniques.html)
  * [RFC 792 - ICMPv4](https://tools.ietf.org/html/rfc792)
  * [RFC 793 - TCP](https://tools.ietf.org/html/rfc793)
  * [Hackplayers - 4nonimizer](https://github.com/Hackplayers/4nonimizer)
  * [Surveillance Self Defense](https://ssd.eff.org/es)
  * [What is a DNS leak and why should I care?](https://www.dnsleaktest.com/what-is-a-dns-leak.html)

<a name="intro"></a>

### 1. Breve introducción informal

#### ICMPv4

Debido a los errores que pueden existir en una red de computadores, necesitamos un modo de saber el estado de los equipos que componen esa red y obtener información para solucionar los problemas que puedan surgir. Ahí es donde entra Internet Control Message Protocol (ICMP). ICMP es un protocolo que nos va a permitir hacer consultas a distintos equipos de una red para conocer su estado. Existen dos versiones de este protocolo:

  * ICMPv4
  * ICMPv6

Nos centraremos en [ICMPv4](https://tools.ietf.org/html/rfc792) porque es el que necesitamos para el contenido del curso. El uso más común del protocolo ICMP es detectar si un equipo esta activo en una red. Lo que coloquialmente llamamos hacer un ping a un equipo. ICMP es un protocolo que opera en la [capa 3](https://es.wikipedia.org/wiki/Capa_de_red) del modelo [OSI](https://es.wikipedia.org/wiki/Modelo_OSI) que funciona enviando peticiones de distintos tipos a los equipos que hay en una red y analizando los tipos de respuesta que recibe. Los tipos de peticiones y de respuesta que se pueden enviar o recibir están unívocamente identificados por un número que podemos consultar en la web de [IANA](https://www.iana.org/assignments/icmp-parameters/icmp-parameters.xhtml). Por ejemplo, el tipo de petición para saber si un equipo está activo en una red es el número 8. Si lanzamos un ping (petición ICMP tipo 8) a un equipo en una red donde el protocolo ICMP esté habilitado y no nos responde, sabremos que el equipo no esta activo. Sin embargo, si el equipo nos responde al ping con un paquete ICMP de tipo 0, sabremos que el equipo está activo en la red.

#### Cabecera del protocolo TCP

Si accedemos a la especificación del protocolo [TCP](https://tools.ietf.org/html/rfc793#page-15) podremos ver la estructura de la cabecera adjunta a todo paquete del protocolo TCP:

<img src="img/tcp_1.png" style="float:left">
<div style="clear:both"></div><br>

Los 6 bits marcados en amarillo corresponden con las flags del protocolo TCP. Estos 6 bits, llamados flags en la cabecera del protocolo TCP, gobiernan la transmisión de datos que se produce en una comunicación a través del protocolo TCP. Con estas flags podemos iniciar, mantener o interrumpir cualquier transmissión a través del protocolo TCP. El detalle de cada flag esta detallado en el temario oficial. Sólo mencionar que, los sistemas operativos basados en [UNIX](https://es.wikipedia.org/wiki/Unix) como GNU/Linux, implementan la pila TCP/IP de acuerdo a esta especificación, lo cuál nos será útil conocerla a la hora de realizar escaneos. Podremos activar y desactivar estos bits llamados flags con el fin de realizar distintos tipos de escaneos. Los sistemas operativos windows no implementan su pila de acuerdo a esta especificación.

<a name="escaneando-red"></a>

### 2. Escaneando la red para descubrir equipos activos

En la fase anterior encontramos el rango de red asociado a la organización u objetivo que se este auditando. Ahora trataremos de descubrir que equipos hay activos en esa red.

#### Detectar si un equipo esta activo usando ping

Podemos usar el comando ping para saber si un equipo esta activo en nuestra red:

<img src="img/ping_1.png" style="float:left">
<div style="clear:both"></div><br>

o en internet:

<img src="img/ping_2.png" style="float:left">
<div style="clear:both"></div><br>

*la opción **-c** es para indicar el número de pings (peticiones ICMP de tipo 8) que queremos enviar*

#### Ping Sweep/ICMP Sweep detectar los equipos activos en una red

Ping Sweep, ICMP Sweep o un barrido de red es hacer un ping a cada equipo de una red para saber si están activos o no. Aunque se podría hacer con un bucle for y analizar la respuesta de cada equipo:

<img src="img/ping_sweep_1.png" style="float:left">
<div style="clear:both"></div><br>

Existen programas que nos van a facilitar esta tarea como veremos a continuación.

#### Ping Sweep/ICMP Sweep usando nmap

Una forma muy sencilla de hacer ping sweep es con nmap. Para ello usamos nmap con la opción *-sn* para deshabilitar el escaneo de puertos y realizar únicamente un ping sweep:

<img src="img/ping_sweep_nmap.png" style="float:left">
<div style="clear:both"></div><br>

como podemos ver, si el protocolo ICMP esta activo, nos detecta todos los equipos activos en esa red.

#### Detectar equipos activos usando netdiscover

Otro programa para detectar los equipos de una red es netdiscover. Esta herramienta funciona esnifando el tráfico ARP. Lo lanzamos usando el comando:

<img src="img/netdiscover_1.png" style="float:left">
<div style="clear:both"></div><br>

y podremos ver los resultados en pantalla:

<img src="img/netdiscover_2.png" style="float:left">
<div style="clear:both"></div><br>

#### Descubriendo equipos activos por TCP usando hping3

Si el protocolo ICMP no estuviese activado en la red que estemos trabajando, podemos usar el protocolo TCP para saber si un equipo esta activo en la red. Un programa con el que podemos hacer esto de forma sencilla es hping3. Hping3 nos va a permitir mandar paquetes TCP con ciertas flags activadas. Por ejemplo, podemos mandar un paquete TCP con la flag *SYN* activada usando la opción *-S*:

<img src="img/hping3_1.png" style="float:left">
<div style="clear:both"></div><br>

*la opción **-c** es para indicar el número de paquetes que queremos enviar*

<a name="detectando-servicios"></a>

### 3. Escaneando los puertos de los equipos activos

Una vez que hemos descubierto los equipos que están activos en una red, intentaremos averiguar qué puertos y qué servicios tienen activos esos equipos.

#### Equipo para realizar escaneos de pruebas (scanme.nmap.org)

El proyecto Nmap ha puesto un equipo en internet para que podamos practicar con nmap de forma responsable. El nombre de dominio es [scanme.nmap.org](http://scanme.nmap.org/):

<img src="img/nmap_equipo_para_practicar.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneando equipos desactivando el host discovery

Nmap, por defecto, detecta los equipos activos en una red lanzando ping (peticiones ICMP de tipo 8). Si el equipo:

  * responde al ping con un paquete ICMP de tipo 0, lo da como activo en la red
  * si no responde al ping lo da como no activo

Si nmap da un equipo por no activo, no realizará más pruebas como un escaneo de puertos. Esto puede ocasionar que, si el protocolo ICMP no esta activado, se den como no activos equipos que si lo están. Por ello, nmap tiene una opción que le podemos pasar para que, en vez de descubrir qué equipos están activos usando ping, directamente dé todos los equipos como activos. De esta forma, podremos escanear los puertos de equipos que tengan bloqueados el tráfico ICMP. Esta opción es *-Pn* y quiere decir (Ping no).

#### Escaneo con handshake completo (fiable pero ruidoso)

Una de las formas que hay de conocer si un puerto TCP esta abierto es realizar el handshake completo. Si podemos realizar el handshake, entonces daremos el puerto como abierto. Es decir, se da el puerto como abierto si:

  1. Enviamos un paquete con la SYN flag activada
  2. Se recibe como respuesta un paquete con las flags SYN y ACK activadas
  3. Enviamos un packete con la flag ACK activada para establecer la conexión
  4. Cortamos la comunicación enviando un paquete con la RST flag activada

Se dice que es ruidoso porque, normalmente, si realizamos un handshake completo, el servidor guardará en sus logs nuestra conexión. La opción para realizar un escaneo con handshake completo es *-sT*. En este caso lanzamos un escaneo contra todos los puertos:

<img src="img/nmap_st_scan.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneo con medio handshake (SYN Scan)

En este tipo de escaneo de puertos del protocolo TCP se hace medio handshake:

  1. Se envia un paquete con la SYN flag activada
  2. Si se recibe como respuesta un paquete con las flags SYN y ACK activadas, se da como activo
  3. Se corta el handshake enviando un paquete con la RST flag activada
  
Muchos servidores no guardarán en sus logs nuestra conexión ya que no se ha producido un handshake completo. Aunque si el la red que estemos auditando cuenta con dispositivos de seguridad como [IDS](https://es.wikipedia.org/wiki/Sistema_de_detección_de_intrusos), nuestra conexión quedará registrada. La opción que usaremos para lanzar un escaneo de este tipo es *-sS*. En este caso, vamos a lanzarlo contra puertos específicos:

<img src="img/nmap_syn_scan.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneo detectando la versión de los servicios

Para detectar la versión de los servicios (programas) que hay corriendo en cada puerto podemos usar la opción *-sV*. Lo que hará nmap es intentar obtener el [banner](https://en.wikipedia.org/wiki/Banner_grabbing) de cada servicio:

<img src="img/nmap_sv_scan_1.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneo de medio handshake detectando la versión de los servicios

Podemos combinar el escaneo SYN Scan con la detección de servicios pasando ambas opciones, *-sS* y *-sV* al lanzar nmap:

<img src="img/nmap_sv_scan_2.png" style="float:left">
<div style="clear:both"></div><br>

o usando la versión abreviada *-sSV*:

<img src="img/nmap_sv_scan_3.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneo de servicios en puertos UDP

La opción para descubrir servicios en puertos del protocolo UDP es *-sU*:

<img src="img/nmap_su_scan.png" style="float:left">
<div style="clear:both"></div><br>

#### Detectando el sistema operativo

Debido a la implementación que cada sistema operativo hace la pila [TCP/IP](https://en.wikipedia.org/wiki/TCP/IP_stack_fingerprinting), en muchas ocasiones, se puede determinar el sistema operativo de un equipo remoto enviando paquetes y analizando valores en los paquetes de respuesta como el ISN, el tamaño de ventana o el TTL. Podemos usar la opción *-O* para detectar el sistema operativo usando nmap:

<img src="img/nmap_os_scan.png" style="float:left">
<div style="clear:both"></div><br>

Otro programa que podemos utilizar para averiguar el sistema operativo es [p0f](http://lcamtuf.coredump.cx/p0f3/). P0f detecta el sistema operativo de forma pasiva esnifando tráfico. Para usarlo, primero debemos indicarle la interfaz de red a usar. Para ver las interfaces de red, por ejemplo, podemos usar el comando *netstat -i*:

<img src="img/netstat_i.png" style="float:left">
<div style="clear:both"></div><br>

Y después usar p0f indicando la interfaz que queremos usar:

<img src="img/p0f_1.png" style="float:left">
<div style="clear:both"></div><br>

Como es una herramienta que captura tráfico se quedará a la escucha. Ahora podemos, por ejemplo, navegar a la dirección web del equipo que queramos saber el sistema operativo:

<img src="img/p0f_2.png" style="float:left">
<div style="clear:both"></div><br>

y localizar en las capturas que hace p0f el sistema operativo:

<img src="img/p0f_3.png" style="float:left">
<div style="clear:both"></div><br>

#### Nmap Scripting Engine (NSE)

Se pueden programs scripts para nmap en el lenguaje de programación [Lua](https://es.wikipedia.org/wiki/Lua). La parte de nmap que ejecuta estos scripts se llama Nmap Scripting Engine (NSE). Aunque nosotros podemos programar nuestros propios scripts, nmap viene de serie con muchos scripts que ya han sido programados por la comunidad de desarrolladores. Podemos utilizar un script en concreto con la opción *--script {nombreDelScript}*. Por ejemplo, podemos usar el script *http-methods* para saber qué métodos del protocolo HTTP el servicio web:

<img src="img/nmap_script_scan_1.png" style="float:left">
<div style="clear:both"></div><br>

También podemos indicarle a nmap que, cuando escane el activo, use todos los scripts que tenga disponible para ese servicio. Para ello usamos la opción *--script=default*:

<img src="img/nmap_script_scan_2.png" style="float:left">
<div style="clear:both"></div><br>

#### Escaneo agresivo con nmap

Podemos pasarle a Nmap la opción *-A* para que haga un escaneo agresivo. Es decir, activará las siguientes opciones:

  * Detección de sistema operativo (-O)
  * Detección de versiones de los servicios (programas) en cada puerto (-sV)
  * Ejecutará scripts para analizar vulnerabilidades en los servicios (--script=default)
  * Activará traceroute (--traceroute)

Esta opción *-A* suele pasarse a nmap junto con un timing agresivo. El timing quiere decir cuánto tiempo va a esperar nmap a que el servidor le responda por cada petición que haga. El valor más bajo es 0 (esperará mucho tiempo) y el valor más alto es 5 (casi no esperará). En este caso usamos la opción *-T* para pasarle un valor de timing:

<img src="img/nmap_a_scan.png" style="float:left">
<div style="clear:both"></div><br>

Aunque hace mucho ruido, los resultados son muy completos. Por ejemplo, se detecta que el servicio ftp tiene habilitado el acceso anónimo. Aunque este timing hace que tarde poco el scan, se recomienda usar valores de timing más bajos para hacer menos ruido.

#### Escaneo ACK para descubrir dispositivos de seguridad

De acuerdo a la especificación del protocolo [TCP](https://tools.ietf.org/html/rfc793), si enviamos un paquete con la flag ACK activada, deberiamos recibir siempre respuesta tanto si el puerto esta abierto como cerrado. Podemos aprovecharnos de esto para intentar detectar dispositivos de seguridad como ciertos tipos de firewall. Es decir, como siempre que enviamos un paquete con la flag ACK activada, deberiamos recibir respuesta tanto si el puerto esta abierto como si no lo está. Si enviamos un paquete con la flag ACK activada y no recibimos respuesta, probablemente, exista algún dispositivo de seguridad. Para lanzar un escaneo ACK con nmap usamos la opción *-sA*:

<img src="img/nmap_sa_scan.png" style="float:left">
<div style="clear:both"></div><br>

También podemos usar hping3 para lanzar un escaneo ACK pasandole activando la flag ACK con la opción *-A*:

<img src="img/hping3_a.png" style="float:left">
<div style="clear:both"></div><br>

<a name="proxies"></a>

### 4. Proxies y programas que ayudan proteger la privacidad e intimidad

La privacidad y el anónimato en internet no se puede proteger sólo con soluciones tecnológicas. La privacidad y anonimato en internet requieren de hábitos y otras conductas que complementen las soluciones técnicas. Grosso modo:

  * La **privacidad** es que nadie sepa qué se está haciendo mientras se está conectado a internet
  * El **anónimato** es que no se sepa quién es la persona que hay detrás de las acciones que se están realizando
  
Cuando queramos anónimato, buscaremos soluciones técnicas como los proxies, cadenas de proxies o redes alternativas como [TOR](https://www.torproject.org/), [I2P](https://geti2p.net/en/), [GNUNet](https://gnunet.org/) o [Freenet](https://freenetproject.org/). Cuando queramos privacidad buscaremos soluciones técnológicas como las [VPN](https://es.wikipedia.org/wiki/Red_privada_virtual). Lo ideal es encontrar un servicio VPN que no registre nuestra actividad en un páis cuyas leyes no obliguen al propietario del servicio VPN a hacerlo. En este tipo de servicios dependemos en gran medida de que el propietario del servicio no registre nuestra actividad. Asimismo las VPN y otros servicios de anónimato están amenazados constantemente por configuraciones de serie que facilitan los proveedores de acceso a internet como los [dnsleaks](https://www.dnsleaktest.com/what-is-a-dns-leak.html). Tambíen estas soluciones tecnológicas se ven amenazadas por la actividad que, en muchas ocasiones, se realiza a través de redes sociales o teléfonos móviles. En muchos casos, para conseguir privacidad y anónimato, se requiere crear una identidad dígital paralela que no tenga relación alguna con la anterior en donde se combine el uso de soluciones técnologicas de privacidad y anonimato. Aunque se sale del objeto de este manual cubrir estas soluciones técnologicas, veremos a continuación dos soluciones que podremos utilizar cuando estemos realizando una auditoria de seguridad a la que nos hayan autorizado.

#### Breve introducción informal a los conceptos de proxy y cadena de proxies

Un **proxy** es un equipo intermedio que se puede actuar cuando queramos acceder a un sitio. Es decir, nosotros nos conectamos al proxy y el proxy se conecta al equipo destino. El equipo destino, en vez de registrar que nos hemos conectado nosotros, registra que se ha conectado el proxy. Los proxies tienen muchos usos como intermediarios. Desde el punto de vista del anonimato nos interesan aquellos proxies (equipos intermedios) que no registren nuestra actividad. Es decir, que mantengan nuestro anonimato. A estos proxies los llamaremos anonimizadores. Una **cadena de proxies** es usar más de un proxy para acceder a un equipo destino. Por ejemplo, si usamos una cadena de dos proxies, primero nos conectamos al primer proxy, el primero se conecta al segundo proxy y el segundo al equipo destino. El equipo destino sólo vé la dirección del segundo proxy. Un programa muy conocido que veremos a continuación para crear cadenas de proxies es [proxychains](http://proxychains.sourceforge.net/).

#### Escaneo con nmap a través de TOR usando proxychains

Si en una auditoria nos bloquean la IP que estemos usando o si simplemente queremos un método sencillo para cambiar la IP desde la que estemos escaneando, podemos usar proxychains para redirigir el tráfico de nuestro equipo a través de una cadena de proxies. En este caso usaremos proxychains para redirigir el tráfico de nuestra máquina a través de la red [TOR](https://en.wikipedia.org/wiki/Tor_%28anonymity_network%29) para cambiar de IP y hacer el escaneo de forma más "anónima". Para ello primero instalamos tor:

<img src="img/nmap_tor_scan_1.png" style="float:left">
<div style="clear:both"></div><br>

y después, si no viniese por defecto instalado, instalamos proxychains:

<img src="img/nmap_tor_scan_2.png" style="float:left">
<div style="clear:both"></div><br>

Si durante la instalación diese algún problema de dependencias, lo solucionamos como de costumbre con:

<img src="img/nmap_tor_scan_3.png" style="float:left">
<div style="clear:both"></div><br>

Una vez instalado, iniciamos el servicio:

<img src="img/nmap_tor_scan_4.png" style="float:left">
<div style="clear:both"></div><br>

Por defecto *proxychains* redirige el tráfico a través de la red TOR. Probamos que la instalación se ha realizado correctamente. Para ello verificamos la IP pública que tenemos, por ejemplo, usando el programa *curl*:

<img src="img/nmap_tor_scan_5.png" style="float:left">
<div style="clear:both"></div><br>

y la que tenemos tras redigir el tráfico a través de TOR usando proxychains:

<img src="img/nmap_tor_scan_6.png" style="float:left">
<div style="clear:both"></div><br>

Podemos usar proxychains con cualquier programa. Por ejemplo, si lo usamos con el navegador firefox:

<img src="img/firefox_tor_1.png" style="float:left">
<div style="clear:both"></div><br>

redirigiremos todo el tráfico a través de TOR y tendremos una nueva IP:

<img src="img/firefox_tor_2.png" style="float:left">
<div style="clear:both"></div><br>

Ahora podemos lanzar nmap de forma "anónima" redirigiendo el tráfico a través de TOR:

<img src="img/nmap_tor_scan_7.png" style="float:left">
<div style="clear:both"></div><br>

#### Hackplayers - 4nonimizer

[4nonimizer](https://github.com/Hackplayers/4nonimizer) nos va a ayudar de forma sencilla a conseguir privacidad mediante el uso de una VPN y de anonimato. En el repositorio de github de **4nonimizer** tienen una descripción muy completa del script:

<img src="img/4nonimizer_1.png" style="float:left">
<div style="clear:both"></div><br>

Vamos a ver cómo podemos usarlo para poder conectar con el servicio VPN que viene por defecto *VPNBook*. Primero tenemos que instalar *4nonimizer* en Kali Linux. Para ello clonamos el repositorio de [github](https://github.com/Hackplayers/4nonimizer.git):

<img src="img/4nonimizer_2.png" style="float:left">
<div style="clear:both"></div><br>

accedemos al directorio del repositorio y tecleamos el comando *./4nonimizer install*:

<img src="img/4nonimizer_3.png" style="float:left">
<div style="clear:both"></div><br>

para instalarlo:

<img src="img/4nonimizer_4.png" style="float:left">
<div style="clear:both"></div><br>

Tras terminar la instalación nos muestra dos opciones que le podemos pasar al comando:

  * start
  * help
  
<img src="img/4nonimizer_5.png" style="float:left">
<div style="clear:both"></div><br>

Antes de comenzar a usarlo, para evitar los errores que puede darnos el script con el DNS de nuestro proveedor de servicios de internet, configuramos el servidor de DNS de Google (8.8.8.8):

<img src="img/4nonimizer_dns.png" style="float:left">
<div style="clear:both"></div><br>

También comprobamos cuál es nuestra IP:

<img src="img/4nonimizer_6.png" style="float:left">
<div style="clear:both"></div><br>

y depués iniciamos el servicio para que se conecte a la VPN con el comando *4nonimizer start*:

<img src="img/4nonimizer_7.png" style="float:left">
<div style="clear:both"></div><br>

Nos conectamos por VPN al proveedor VPNBook y se nos asigna una nueva dirección IP:

<img src="img/4nonimizer_8.png" style="float:left">
<div style="clear:both"></div><br>

Confirmamos la IP:

<img src="img/4nonimizer_9.png" style="float:left">
<div style="clear:both"></div><br>

y vemos que coincide. Si quisieramos cambiar la IP, podemos hacerlo con el comando *4nonimizer change_ip*:

<img src="img/4nonimizer_10.png" style="float:left">
<div style="clear:both"></div><br>

y el servicio VPN nos facilitaría una nueva IP:

<img src="img/4nonimizer_11.png" style="float:left">
<div style="clear:both"></div><br>

Ahora que estamos conectado a la VPN y hemos conseguido privacidad, si además quisiéramos añadir anonimato, podemos lanzar nuestro escaneo de nmap a través de la red TOR usando proxychains:

<img src="img/4nonimizer_12.png" style="float:left">
<div style="clear:both"></div><br>

Para detener el servicio, usamos el comando *4nonimizer stop*:

<img src="img/4nonimizer_13.png" style="float:left">
<div style="clear:both"></div><br>

Como casi siempre para ampliar información, podemos usar el comando de ayuda *4nonimizer help*:

<img src="img/4nonimizer_14.png" style="float:left">
<div style="clear:both"></div><br>

Para ver todas la opciones del programa:

<img src="img/4nonimizer_15.png" style="float:left">
<div style="clear:both"></div><br>

<a name="detectando-vulnerabilidades"></a>

### 5. Escaneando en busca de vulnerabilidades conocidas

Tras conocer los puertos abiertos, servicios y sistemas operativos que tienen los activos que se estén auditando, podemos analizarlos con un escaner de vulnerabilidades para detectar si hubiese fallos en la configuración de los equipos o si las versiones de los sistemas operativos o de los servicios (programas) que se están usando estuviesen afectadas por vulnerabilidades conocidas.

#### Breve introducción informal sobre los escáneres de vulnerabilidades

Los escáneres de vulnerabilidades son programas diseñados para analizar si un equipo o una red de equipos no están configurados correctamente o si las versiones de los programas que usan los servicios tienen vulnerabilidades conocidas que, potencialmente, podrían ser aprovechadas por un atacante para comprometer la confidencialidad, integridad o disponibilidad de los activos. Un escaner de vulnerabilidades es un programa informático que esta compuesto, principalmente, de 3 partes:

  * Una base de datos actualizada que contiene las vulnerabilidades conocidas.
  * Un programa, normalmente llamado escáner, que prueba si las vulnerabilidades que hay en la base de datos están presentes en el equipo que le indiquemos. También puede comprobar configuraciones inseguras como dejar contraseñas por defecto.
  * Una página web para que el usuario pueda configurar los escaneos de vulnerabilidades.
  
Algunos conocidos son:
  
  * [Acunetix](https://www.acunetix.com/)
  * [Nessus](https://www.tenable.com/products/nessus/nessus-professional)
  * [Nexpose](https://www.rapid7.com/products/nexpose/)
  * [OpenVas](http://www.openvas.org/)

#### Breve introducción informal sobre el análisis de vulnerabilidades

Un análisis de vulnerabilidades es un servicio que comprende 3 partes:

  1. Escanear un equipo o varios equipos utilizando un escáner de vulnerabilidades
  2. Determinar qué resultados son falsos positivos y qué resultados son vulnerabilidades reales
  3. Reportar las vulnerabilidades reales

Un escáner de vulnerabilidades arroja muchos falsos positivos que deben ser verificados para que sólo se reporten las vulnerabilidades reales. La verificación de los resultados de un escaner de vulnerabilidades, normalmente, suelen realizarla equipos especializados en el análisis de vulnerabilidades.

#### Breve Introducción al análisis de vulnerabilidades con OpenVas

Se ha elegido [OpenVas](http://www.openvas.org/) porque es [software libre](https://directory.fsf.org/wiki/OpenVAS). Para usar OpenVas con Kali Linux, primero debemos instalarlo:

<img src="img/openvas_1.png" style="float:left">
<div style="clear:both"></div><br>

Una vez que este instalado. Debemos ejecutar el siguiente comando para configurarlo:

<img src="img/openvas_2.png" style="float:left">
<div style="clear:both"></div><br>

tardará bastante en ejecutarse porque, entre otros, tiene que actualizar la base de datos de vulnerabilidades conocidas y generar los certificados que se usuarán en la página web local desde la que administraremos los escaneos. Una vez termine habrá creado un usuario llamado *admin* y nos facilitará la contraseña por pantalla:

<img src="img/openvas_3.png" style="float:left">
<div style="clear:both"></div><br>

Copiamos la contraseña e iniciamos OpenVas con el comando:

<img src="img/openvas_4.png" style="float:left">
<div style="clear:both"></div><br>

Una vez iniciado OpenVas, abrimos el navegador y accedemos a la siguiente URL: https://localhost:9392. Debido a que el certificado no es reconocido por el navegador, tendremos que hacer click en el botón *Advanced*:

<img src="img/openvas_5.png" style="float:left">
<div style="clear:both"></div><br>

a continuación hacer click sobre el botón *Add Exception*:

<img src="img/openvas_6.png" style="float:left">
<div style="clear:both"></div><br>

y finalmente pulsar sobre el botón *Confirm Security Exception*:

<img src="img/openvas_7.png" style="float:left">
<div style="clear:both"></div><br>

Nada más acceder a la página web de administración de OpenVas, tendremos que logarnos con el usuario *admin* y la contraseña que se nos facilitó al ejecutar el comando *openvas-setup*:

<img src="img/openvas_8.png" style="float:left">
<div style="clear:both"></div><br>

Una vez logados, nos aparecerá un pequeño Dashboard donde figurarán los resúmenes de los escaneos que realicemos. Para comenzar, de forma rápida, a escanear un equipo, hacemos click en el menú *Scans* y seleccionamos la opción *Tasks*:

<img src="img/openvas_9.png" style="float:left">
<div style="clear:both"></div><br>

En esta pestaña, hacemos click en el icono de color morado y seleccionamos la opción *Task Wizard*:

<img src="img/openvas_10.png" style="float:left">
<div style="clear:both"></div><br>

Aparecerá el asistente para la configuración rápida de un nuevo escaneo. En este caso escaneamos una IP de una máquina virtual [(metasploitable 2)](https://sourceforge.net/projects/metasploitable/files/Metasploitable2/) y pulsamos el botón *Start Scan*:

<img src="img/openvas_11.png" style="float:left">
<div style="clear:both"></div><br>

y comenzará el escaneo. Si, al final de la pantalla, pulsamos sobre el escaneo que está ejecutandose:

<img src="img/openvas_12.png" style="float:left">
<div style="clear:both"></div><br>

Podremos ver los detalles del escaneo en curso y acceder a los resultados y al reporte en curso:

<img src="img/openvas_13.png" style="float:left">
<div style="clear:both"></div><br>

Una vez que el escaneo haya terminado, si volvemos al dashboard, vermos que nuestro escaneo queda reflejado:

<img src="img/openvas_14.png" style="float:left">
<div style="clear:both"></div><br>

Para ver los resultados del scan, seleccionamos la opción *Results* del menu Scans:

<img src="img/openvas_15.png" style="float:left">
<div style="clear:both"></div><br>

El escaneo ha encontrado 335 vulnerabilidades en el equipo analizado. También se muestran representaciones gráficas de los resultados para que rápidamente podamos ver el número de vulnerabilidades altas, medias y bajas que se han encontrado. Si pulsamos sobre cualquiera de ellas, podremos ver el detalle:

<img src="img/openvas_16.png" style="float:left">
<div style="clear:both"></div><br>

En este caso nos reporta que se ha encontrado un servicio SSH con contraseñas por defecto y nos muestra los usuarios utilizados para conectarse:

<img src="img/openvas_17.png" style="float:left">
<div style="clear:both"></div><br>

Comprobamos si la vulnerabilidad reportada es real o un falso positivo. Para ello, nos conectamos por ssh al equipo utilizando una de las credenciales por defecto. Por ejemplo:

  * Usuario: user
  * Contraseña: user
  
<img src="img/openvas_18.png" style="float:left">
<div style="clear:both"></div><br>

Y vemos que podemos logarnos en la máquina por ssh con esas credenciales. Luego en este caso, sería un vulnerabilidad real que se debe a una configuración incorrecta del equipo.